<a href="https://colab.research.google.com/github/ivikasavnish/forum-notebook/blob/main/nse500_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NSE 500 Stock Analysis - 1 Year Data

This notebook allows you to:
1. Select stocks from NSE 500
2. Collect 1 year of **hourly** (intraday) data
3. Collect 1 year of **daily** data
4. Find the most common up and down percentage changes (rounded to nearest 0.5%)

**Note**: Yahoo Finance API limits true minute-level (1m) data to the last 7 days only. For 1-year intraday analysis, this notebook uses hourly (1h) intervals.

In [10]:
# Install required packages
!pip install yfinance pandas numpy matplotlib seaborn -q

In [11]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

Libraries imported successfully!


In [12]:
# NSE 500 stocks (sample list with popular stocks)
# Note: For yfinance, NSE stocks need .NS suffix
NSE500_STOCKS = [
    "RELIANCE.NS", "TCS.NS", "HDFCBANK.NS", "INFY.NS", "ICICIBANK.NS",
    "HINDUNILVR.NS", "ITC.NS", "SBIN.NS", "BHARTIARTL.NS", "BAJFINANCE.NS",
    "KOTAKBANK.NS", "LT.NS", "ASIANPAINT.NS", "AXISBANK.NS", "MARUTI.NS",
    "HCLTECH.NS", "WIPRO.NS", "ULTRACEMCO.NS", "TITAN.NS", "SUNPHARMA.NS",
    "NESTLEIND.NS", "BAJAJFINSV.NS", "TATAMOTORS.NS", "ONGC.NS", "NTPC.NS",
    "TECHM.NS", "M&M.NS", "POWERGRID.NS", "ADANIPORTS.NS", "TATASTEEL.NS",
    "HINDALCO.NS", "JSWSTEEL.NS", "INDUSINDBK.NS", "DIVISLAB.NS", "DRREDDY.NS",
    "CIPLA.NS", "EICHERMOT.NS", "BRITANNIA.NS", "COALINDIA.NS", "GRASIM.NS",
    "APOLLOHOSP.NS", "BPCL.NS", "HEROMOTOCO.NS", "SHREECEM.NS", "UPL.NS",
    "ADANIENT.NS", "BAJAJ-AUTO.NS", "SBILIFE.NS", "HDFCLIFE.NS", "TATACONSUM.NS"
]

# Create a cleaner display list (without .NS suffix)
DISPLAY_NAMES = [stock.replace('.NS', '') for stock in NSE500_STOCKS]

print(f"Loaded {len(NSE500_STOCKS)} NSE stocks")
print("\nSample stocks:", DISPLAY_NAMES[:10])

Loaded 50 NSE stocks

Sample stocks: ['RELIANCE', 'TCS', 'HDFCBANK', 'INFY', 'ICICIBANK', 'HINDUNILVR', 'ITC', 'SBIN', 'BHARTIARTL', 'BAJFINANCE']


In [13]:
# Stock selection using dropdown
from ipywidgets import Dropdown, Button, VBox, HBox, Output
from IPython.display import display, clear_output

# Create dropdown widget
stock_dropdown = Dropdown(
    options=[(name, ticker) for name, ticker in zip(DISPLAY_NAMES, NSE500_STOCKS)],
    value=NSE500_STOCKS[0],
    description='Select Stock:',
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)

# Create button to fetch data
fetch_button = Button(
    description='Fetch Data',
    button_style='primary',
    layout={'width': '150px'}
)

# Output widget for messages
output = Output()

# Display widgets
display(VBox([HBox([stock_dropdown, fetch_button]), output]))

print("Stock selection widget ready!")

Stock selection widget ready!


In [14]:
# Functions to collect data
def collect_minute_data(ticker, period="1y"):
    """
    Collect hourly intraday data for the specified ticker.
    Note: Yahoo Finance API limits 1-minute data to 7 days.
    For 1 year of intraday data, we use hourly (1h) intervals.
    """
    print(f"Fetching hourly (intraday) data for {ticker}...")
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period="1y", interval="1h")

        if df.empty:
            print(f"No data available for {ticker}")
            return None

        print(f"✓ Collected {len(df)} hourly data points")
        return df
    except Exception as e:
        print(f"Error fetching hourly data: {e}")
        return None

def collect_daily_data(ticker, period="1y"):
    """
    Collect daily data for the specified ticker.
    """
    print(f"Fetching daily data for {ticker}...")
    try:
        stock = yf.Ticker(ticker)
        df = stock.history(period=period)

        if df.empty:
            print(f"No data available for {ticker}")
            return None

        print(f"✓ Collected {len(df)} daily data points")
        return df
    except Exception as e:
        print(f"Error fetching daily data: {e}")
        return None

print("Data collection functions defined!")

Data collection functions defined!


In [15]:
# Functions to calculate percentage changes
def calculate_percentage_changes(df):
    """
    Calculate percentage changes and round to nearest 0.5%
    """
    if df is None or df.empty:
        return None, None

    # Calculate percentage change
    df['Pct_Change'] = df['Close'].pct_change() * 100

    # Remove NaN values
    pct_changes = df['Pct_Change'].dropna()

    # Round to nearest 0.5%
    rounded_changes = (pct_changes / 0.5).round() * 0.5

    # Separate into ups and downs
    ups = rounded_changes[rounded_changes > 0]
    downs = rounded_changes[rounded_changes < 0]

    return ups, downs

def find_most_common(data, n=10):
    """
    Find the most common values in the data
    """
    if data is None or len(data) == 0:
        return []

    counter = Counter(data)
    return counter.most_common(n)

print("Percentage calculation functions defined!")

Percentage calculation functions defined!


In [16]:
# Visualization functions
def plot_common_changes(ups_common, downs_common, title_suffix=""):
    """
    Plot the most common percentage changes
    """
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    # Plot ups
    if ups_common:
        ups_values = [x[0] for x in ups_common]
        ups_counts = [x[1] for x in ups_common]

        ax1.barh(range(len(ups_values)), ups_counts, color='green', alpha=0.7)
        ax1.set_yticks(range(len(ups_values)))
        ax1.set_yticklabels([f"{v:+.1f}%" for v in ups_values])
        ax1.set_xlabel('Frequency')
        ax1.set_title(f'Most Common UP Percentages {title_suffix}')
        ax1.grid(axis='x', alpha=0.3)

        for i, count in enumerate(ups_counts):
            ax1.text(count, i, f' {count}', va='center')
    else:
        ax1.text(0.5, 0.5, 'No up movements', ha='center', va='center')
        ax1.set_xlim(0, 1)

    # Plot downs
    if downs_common:
        downs_values = [x[0] for x in downs_common]
        downs_counts = [x[1] for x in downs_common]

        ax2.barh(range(len(downs_values)), downs_counts, color='red', alpha=0.7)
        ax2.set_yticks(range(len(downs_values)))
        ax2.set_yticklabels([f"{v:+.1f}%" for v in downs_values])
        ax2.set_xlabel('Frequency')
        ax2.set_title(f'Most Common DOWN Percentages {title_suffix}')
        ax2.grid(axis='x', alpha=0.3)

        for i, count in enumerate(downs_counts):
            ax2.text(count, i, f' {count}', va='center')
    else:
        ax2.text(0.5, 0.5, 'No down movements', ha='center', va='center')
        ax2.set_xlim(0, 1)

    plt.tight_layout()
    plt.show()

def display_statistics(ups, downs, title_suffix=""):
    """
    Display statistics about the percentage changes
    """
    print(f"\n{'='*60}")
    print(f"Statistics {title_suffix}")
    print(f"{'='*60}")

    if len(ups) > 0:
        print(f"\n📈 UP Movements:")
        print(f"  Total count: {len(ups)}")
        print(f"  Average: {ups.mean():.2f}%")
        print(f"  Median: {ups.median():.2f}%")
        print(f"  Max: {ups.max():.2f}%")

    if len(downs) > 0:
        print(f"\n📉 DOWN Movements:")
        print(f"  Total count: {len(downs)}")
        print(f"  Average: {downs.mean():.2f}%")
        print(f"  Median: {downs.median():.2f}%")
        print(f"  Min: {downs.min():.2f}%")

    print(f"\n{'='*60}\n")

print("Visualization functions defined!")

Visualization functions defined!


In [17]:
# Main analysis function
def analyze_stock(ticker):
    """
    Perform complete analysis for the selected stock
    """
    stock_name = ticker.replace('.NS', '')
    print(f"\n{'='*60}")
    print(f"Analyzing {stock_name}")
    print(f"{'='*60}\n")

    # Collect minute-level data (hourly as compromise)
    print("\n1️⃣  HOURLY DATA ANALYSIS (Intraday - 1 Year)")
    print("-" * 60)
    minute_data = collect_minute_data(ticker)

    if minute_data is not None:
        ups_minute, downs_minute = calculate_percentage_changes(minute_data)

        if ups_minute is not None and downs_minute is not None:
            # Find most common
            ups_common_minute = find_most_common(ups_minute, n=10)
            downs_common_minute = find_most_common(downs_minute, n=10)

            # Display statistics
            display_statistics(ups_minute, downs_minute, "(Hourly)")

            # Print most common
            print("\nMost Common UP Percentages (Hourly):")
            for pct, count in ups_common_minute[:5]:
                print(f"  {pct:+.1f}%: {count} times")

            print("\nMost Common DOWN Percentages (Hourly):")
            for pct, count in downs_common_minute[:5]:
                print(f"  {pct:+.1f}%: {count} times")

            # Plot
            plot_common_changes(ups_common_minute, downs_common_minute, "(Hourly)")

    # Collect daily data
    print("\n2️⃣  DAILY DATA ANALYSIS (1 Year)")
    print("-" * 60)
    daily_data = collect_daily_data(ticker)

    if daily_data is not None:
        ups_daily, downs_daily = calculate_percentage_changes(daily_data)

        if ups_daily is not None and downs_daily is not None:
            # Find most common
            ups_common_daily = find_most_common(ups_daily, n=10)
            downs_common_daily = find_most_common(downs_daily, n=10)

            # Display statistics
            display_statistics(ups_daily, downs_daily, "(Daily)")

            # Print most common
            print("\nMost Common UP Percentages (Daily):")
            for pct, count in ups_common_daily[:5]:
                print(f"  {pct:+.1f}%: {count} times")

            print("\nMost Common DOWN Percentages (Daily):")
            for pct, count in downs_common_daily[:5]:
                print(f"  {pct:+.1f}%: {count} times")

            # Plot
            plot_common_changes(ups_common_daily, downs_common_daily, "(Daily)")

    print(f"\n{'='*60}")
    print(f"Analysis complete for {stock_name}!")
    print(f"{'='*60}\n")

print("Main analysis function defined!")

Main analysis function defined!


In [18]:
# Button click handler
def on_fetch_button_clicked(b):
    with output:
        clear_output()
        selected_ticker = stock_dropdown.value
        analyze_stock(selected_ticker)

fetch_button.on_click(on_fetch_button_clicked)

print("\n✅ Setup complete! Select a stock and click 'Fetch Data' to begin analysis.")


✅ Setup complete! Select a stock and click 'Fetch Data' to begin analysis.


## Usage Instructions

1. **Select a Stock**: Use the dropdown menu to select a stock from the NSE 500 list
2. **Fetch Data**: Click the "Fetch Data" button to start the analysis
3. **View Results**: The notebook will:
   - Fetch 1 year of hourly data (intraday)
   - Fetch 1 year of daily data
   - Calculate percentage changes
   - Round changes to nearest 0.5%
   - Display the most common up and down percentages
   - Show visualizations of the results

### Note on Data Limitations

- **Minute Data**: yfinance API limits minute-level data to the last 7 days only. Therefore, for 1-year intraday analysis, we use **hourly intervals** as the best available alternative.
- **Daily Data**: Full 1 year of daily data is available and used for analysis.

### Sample Stocks Included

The notebook includes 50 popular NSE stocks including:
- Large caps: RELIANCE, TCS, HDFCBANK, INFY, ICICIBANK
- Banking: SBIN, KOTAKBANK, AXISBANK, INDUSINDBK
- IT: TCS, INFY, WIPRO, HCLTECH, TECHM
- And many more...

### Customization

You can easily add more stocks to the `NSE500_STOCKS` list by adding tickers with the `.NS` suffix (e.g., "STOCKNAME.NS").